In [1]:
import findspark
findspark.init('/home/srabin/spark-3.5.1-bin-hadoop3')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp_exe').getOrCreate()

24/08/08 14:21:12 WARN Utils: Your hostname, DESKTOP-P0KTK6U resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/08/08 14:21:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/08 14:21:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/08 14:21:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/08/08 14:21:14 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/08/08 14:21:14 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/08/08 14:21:14 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/08/08 14:21:14 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port

In [4]:
data = spark.read.csv('SMSSpamCollection', inferSchema= True, sep = '\t') #seperated by tab not comma

In [5]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [7]:
# relabel the columns
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text') 

In [8]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [9]:
from pyspark.sql.functions import length

In [10]:
data = data.withColumn('length', length(data['text'])) # create another data

In [11]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [12]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [13]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [14]:
tokenizer = Tokenizer (inputCol = 'text', outputCol = 'token_text')

In [15]:
stop_remove = StopWordsRemover(inputCol = 'token_text', outputCol = 'stop_token')

In [16]:
count_vec = CountVectorizer(inputCol = 'stop_token', outputCol = 'c_vec')

In [17]:
idf = IDF (inputCol = 'c_vec', outputCol = 'tf_idf')

In [18]:
ham_spam_to_numeric = StringIndexer(inputCol = 'class', outputCol = 'label')

In [19]:
from pyspark.ml.feature import VectorAssembler

In [33]:
clean_up = VectorAssembler(inputCols = ['tf_idf', 'length'],outputCol = 'features')

In [34]:
from pyspark.ml.classification import NaiveBayes

In [35]:
nb = NaiveBayes()

In [36]:
from pyspark.ml import Pipeline

In [37]:
data_prep_pipe = Pipeline(stages = [ham_spam_to_numeric, tokenizer, stop_remove, count_vec, idf, clean_up])

In [38]:
cleaner = data_prep_pipe.fit(data)

In [39]:
clean_data = cleaner.transform(data)

In [41]:
clean_data = clean_data.select('label', 'features')

In [43]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,140...|
|  0.0|(13424,[10,53,102...|
|  0.0|(13424,[127,185,4...|
|  1.0|(13424,[1,47,121,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,117...|
|  1.0|(13424,[8,16,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,95,221...|
|  0.0|(13424,[555,1797,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,44...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [44]:
training, test = clean_data.randomSplit([0.7, 0.3])

In [45]:
spam_detector = nb.fit(training)

24/08/08 14:44:59 WARN DAGScheduler: Broadcasting large task binary with size 1166.7 KiB
24/08/08 14:45:00 WARN DAGScheduler: Broadcasting large task binary with size 1146.5 KiB


In [46]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [47]:
test_results = spam_detector.transform(test)

In [48]:
test_results.show()

24/08/08 14:45:38 WARN DAGScheduler: Broadcasting large task binary with size 1371.7 KiB


+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,13,...|[-622.27002638105...|[0.99999999999799...|       0.0|
|  0.0|(13424,[0,1,3,9,1...|[-576.46533093299...|[0.99999999999163...|       0.0|
|  0.0|(13424,[0,1,4,50,...|[-840.86815673367...|[1.0,4.6003735635...|       0.0|
|  0.0|(13424,[0,1,5,15,...|[-1000.6688647395...|[1.0,1.3286748449...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[-655.74142278421...|[1.0,7.4400850882...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-983.38026629427...|[0.99999999999999...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-750.51176850424...|[1.0,7.2734765447...|       0.0|
|  0.0|(13424,[0,1,23,63...|[-1313.5206915781...|[1.0,1.7207134711...|       0.0|
|  0.0|(13424,[0,1,30,11...|[-594.96416085947...|[1.0,1.1428209212...|       0.0|
|  0.0|(13424,[0

24/08/08 14:45:39 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [49]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [50]:
acc_eval = MulticlassClassificationEvaluator()

In [51]:
acc = acc_eval.evaluate(test_results)

24/08/08 14:46:25 WARN DAGScheduler: Broadcasting large task binary with size 1376.8 KiB


In [53]:
print('ACC of NB Model:')
print(acc)

ACC of NB Model:
0.9174245384144645
